# **Numerical Integral Estimator**

## **Contents**
- [Introduction](#Introduction)
- [The Midpoint Rule](#The-Midpoint-Rule)
- [The Trapezoidal Rule](#The-Trapezoidal-Rule)
- [Simpson's Rule](#Simpson's-Rule)
- [Monte Carlo Method](#Monte-Carlo-Method)
- [Credits](#Credits)


## **Introduction**

This project aims to implement standard numerical techniques for computing definite integrals in Python. In particular, the midpoint, trapezoidal and Simpson's rules, as well as the Monte Carlo method from standard continuous uniform random variables are implemented using the `numpy` and `sympy` libraries. A random seed value will be set for reproducibiity. The `time` library is also imported in order to display the computation time of each implementation, as well as the number of iterations per second excuted. In what follows, the following integral will be approximated as an example:
$$\int_{-100}^{100}e^{-x^2}\mathrm{d}x$$
which, up to nine decimal places, satisfies:
$$\int_{-100}^{100}e^{-x^2}\mathrm{d}x \approx \int_{-\infty}^{+\infty}e^{-x^2}\mathrm{d}x = \sqrt{\pi} \approx 1.772453851$$

In [22]:
import time as tm
import numpy as np
import sympy as smp

np.random.seed(30)

x = smp.symbols("x")
f = smp.exp(-x**2)
a = -100
b = 100
n = 1000000

## **The Midpoint Rule**
Let $(a,b)$ be an ordered pair of real numbers in $\mathbb{R}^2$ such that $a < b$, let $f$ be a continuous function in $\mathscr{C}\left([a,b],\mathbb{R}\right)$ and let $n$ be a non-zero natural number in $\mathbb{N}^{*}$. The *midpoint rule approximation of $\int_{a}^{b}f(x)\mathrm{d}x$ of order $n$* is the real number denoted by $\mathrm{M}_{n, [a,b]}(f)$ and defined by
$$\mathrm{M}_{n, [a,b]}(f) = \frac{b - a}{n}\sum_{k=1}^{n}f\left(a + \frac{(b-a)(2k-1)}{2n}\right)$$
It can be proven that:
$$\lim_{n \to +\infty}\left(\mathrm{M}_{n, [a,b]}(f)\right) = \int_{a}^{b}f(x)\mathrm{d}x$$
although the proof is omitted. The Python implementation of the midpoint rule is given in the code cell below, alongside the calculated value and computation time.

In [23]:
def midpoint_rule(f, a, b, n, show_performance_metrics = False):
    x = smp.symbols("x")
    assert a < b and type(n) is int
    g = smp.lambdify(x, f, 'numpy')
    start = tm.time()
    g = np.vectorize(g)
    dx = (b - a) / n
    P = np.linspace(a, b, n + 1)
    S = (P + np.diff(P)[0] / 2)[:-1]
    estimate = float(dx * np.sum(g(S)))
    end = tm.time()
    computation_time = end - start
    if computation_time == 0.0:
        iterations_per_second = "\u221e"
    else:
        iterations_per_second = f"{int(n / computation_time):,}"
    integral = smp.integrate(f, (x, a, b)).evalf()
    if show_performance_metrics:
        print(f"Function: {f}")
        print(f"Domain: [{a}, {b}]")
        print(f"Evaluate: {smp.printing.latex(smp.Integral(f, (x, a, b)))}")
        print(f"Method: {midpoint_rule.__name__}")
        print(f"Number of iterations: {n:,}")
        print(f"Estimate: {estimate}")
        print(f"Exact Value: {integral}")
        print(f"Relative error: {100 * (estimate - integral) / integral:.2f}%")
        print(f"Computation time: {computation_time:.3f} seconds")
        print(f"Iterations per second: {iterations_per_second}")
    else:
        pass
    return estimate

midpoint_rule.__name__ = "Midpoint Rule"

midpoint_rule(f, a, b, n, True)

Function: exp(-x**2)
Domain: [-100, 100]
Evaluate: \int\limits_{-100}^{100} e^{- x^{2}}\, dx
Method: Midpoint Rule
Number of iterations: 1,000,000
Estimate: 1.772453850905516
Exact Value: 1.77245385090552
Relative error: 0.00%
Computation time: 2.526 seconds
Iterations per second: 395,899


1.772453850905516

## **The Trapezoidal Rule**
Let $(a,b)$ be an ordered pair of real numbers in $\mathbb{R}^2$ such that $a < b$, let $f$ be a continuous function in $\mathscr{C}\left([a,b],\mathbb{R}\right)$ and let $n$ be a non-zero natural number in $\mathbb{N}^{*}$. The *trapezoidal rule approximation of $\int_{a}^{b}f(x)\mathrm{d}x$ of order $n$* is the real number denoted by $\mathrm{T}_{n, [a,b]}(f)$ and defined by
$$\mathrm{T}_{n, [a,b]}(f) = \frac{b-a}{2n}\left(f(a)+2\sum_{k=1}^{n-1}f\left(a+\frac{(b-a)k}{n}\right)+f(b)\right)$$
It can be proven that:
$$\lim_{n \to +\infty}\left(\mathrm{T}_{n, [a,b]}(f)\right) = \int_{a}^{b}f(x)\mathrm{d}x$$
although the proof is omitted. The Python implementation of the trapezoidal rule is given in the code cell below, alongside the calculated value and computation time.

In [24]:
def trapezoidal_rule(f, a, b, n, show_performance_metrics = False):
    x = smp.symbols("x")
    assert a < b and type(n) is int
    g = smp.lambdify(x, f, 'numpy')
    start = tm.time()
    g = np.vectorize(g)
    dx = (b - a) / n
    P = np.linspace(a, b, n + 1)
    S = (P + np.diff(P)[0] / 2)[:-1]
    estimate = float(dx * np.sum(g(S)))
    F = g(P)
    estimate = (dx / 2) * np.sum(np.concatenate(([F[0]], 2 * F[1:-1], [F[-1]])))
    end = tm.time()
    computation_time = end - start
    if computation_time == 0.0:
        iterations_per_second = "\u221e"
    else:
        iterations_per_second = f"{int(n / computation_time):,}"
    integral = smp.integrate(f, (x, a, b)).evalf()
    if show_performance_metrics:
        print(f"Function: {f}")
        print(f"Domain: [{a}, {b}]")
        print(f"Evaluate: {smp.printing.latex(smp.Integral(f, (x, a, b)))}")
        print(f"Method: {trapezoidal_rule.__name__}")
        print(f"Number of iterations: {n:,}")
        print(f"Estimate: {estimate}")
        print(f"Exact Value: {integral}")
        print(f"Relative error: {100 * (estimate - integral) / integral:.2f}%")
        print(f"Computation time: {computation_time:.3f} seconds")
        print(f"Iterations per second: {iterations_per_second}")
    else:
        pass
    return estimate

trapezoidal_rule.__name__ = "Trapezoidal Rule"

trapezoidal_rule(f, a, b, n, True)

Function: exp(-x**2)
Domain: [-100, 100]
Evaluate: \int\limits_{-100}^{100} e^{- x^{2}}\, dx
Method: Trapezoidal Rule
Number of iterations: 1,000,000
Estimate: 1.772453850905516
Exact Value: 1.77245385090552
Relative error: 0.00%
Computation time: 5.563 seconds
Iterations per second: 179,771


1.772453850905516

## **Simpson's Rule**
Let $(a,b)$ be an ordered pair of real numbers in $\mathbb{R}^2$ such that $a < b$, let $f$ be a continuous function in $\mathscr{C}\left([a,b],\mathbb{R}\right)$ and let $n$ be a non-zero natural number in $\mathbb{N}^{*}$. The *Simpson's rule approximation of $\int_{a}^{b}f(x)\mathrm{d}x$ of order $n$* is the real number denoted by $\mathrm{S}_{n, [a,b]}(f)$ and defined by
$$\mathrm{S}_{n, [a,b]}(f) = \frac{b-a}{3n}\left(f(a)+4\sum_{k=1}^{\lfloor n/2\rfloor}f\left(a + \frac{(b-a)(2k-1)}{n}\right)+2\sum_{k=1}^{\lfloor n/2\rfloor - 1}f\left(a + \frac{2(b-a)k}{n}\right)+f(b)\right)$$
It can be proven that:
$$\lim_{n \to +\infty}\left(\mathrm{S}_{n, [a,b]}(f)\right) = \int_{a}^{b}f(x)\mathrm{d}x$$
although the proof is omitted. The Python implementation of Simpson's rule is given in the code cell below, alongside the calculated value and computation time.

In [25]:
def simpsons_rule(f, a, b, n, show_performance_metrics = True):
    x = smp.symbols("x")
    assert a < b and type(n) is int
    g = smp.lambdify(x, f, 'numpy')
    start = tm.time()
    g = np.vectorize(g)
    dx = (b - a) / n
    P = np.linspace(a, b, n + 1)
    S = (P + np.diff(P)[0] / 2)[:-1]
    estimate = float(dx * np.sum(g(S)))
    F = g(P)
    estimate = (dx / 3) * np.sum(np.concatenate(([F[0]], 4 * F[1:n:2], 2 * F[2:n-1:2], [F[-1]])))
    end = tm.time()
    computation_time = end - start
    if computation_time == 0.0:
        iterations_per_second = "\u221e"
    else:
        iterations_per_second = f"{int(n / computation_time):,}"
    integral = smp.integrate(f, (x, a, b)).evalf()
    if show_performance_metrics:
        print(f"Function: {f}")
        print(f"Domain: [{a}, {b}]")
        print(f"Evaluate: {smp.printing.latex(smp.Integral(f, (x, a, b)))}")
        print(f"Method: {simpsons_rule.__name__}")
        print(f"Number of iterations: {n:,}")
        print(f"Estimate: {estimate}")
        print(f"Exact Value: {integral}")
        print(f"Relative error: {100 * (estimate - integral) / integral:.2f}%")
        print(f"Computation time: {computation_time:.3f} seconds")
        print(f"Iterations per second: {iterations_per_second}")
    else:
        pass
    return estimate

simpsons_rule.__name__ = "Simpson's Rule"

simpsons_rule(f, a, b, n, True)

Function: exp(-x**2)
Domain: [-100, 100]
Evaluate: \int\limits_{-100}^{100} e^{- x^{2}}\, dx
Method: Simpson's Rule
Number of iterations: 1,000,000
Estimate: 1.7724538509055159
Exact Value: 1.77245385090552
Relative error: -0.00%
Computation time: 5.337 seconds
Iterations per second: 187,357


1.7724538509055159

## **Monte Carlo Method**
Let $(a,b)$ be an ordered pair of real numbers in $\mathbb{R}^2$ such that $a < b$, let $f$ be a continuous function in $\mathscr{C}\left([a,b],\mathbb{R}\right)$ and let $n$ be a non-zero natural number in $\mathbb{N}^{*}$, let $(\Omega, \mathscr{F}, \mathbb{P})$ be a probability space and let $U$ be a standard continuous uniform random variable on $(\Omega, \mathscr{F}, \mathbb{P})$, that is, a random variable on $(\Omega, \mathscr{F}, \mathbb{P})$ such that $U \hookrightarrow \mathsf{ContinuousUniform}\left(0, 1\right)$. Then it follows by $\left(\mathscr{B}([a,b]),\mathscr{B}(\mathbb{R})\right)$-measurability (i.e., Borel measurability) of $f$ (which itself follows from continuity of $f$) and by the law of the subconscious statistician that $(b - a)f\left(a + (b - a)U\right)$ is a well-defined random variable on $(\Omega,\mathscr{F},\mathbb{P})$ and that one has:
$$\mathbb{E}\left((b - a)f\left(a + (b - a)U\right)\right) = \int_{a}^{b}f(x)\mathrm{d}x$$
Thus, from the strong law of large numbers that, if $(U_{n})_{n \in \mathbb{N}}$ is an independent and identically distributed sequence of standard continuous uniform random variables, then $\left((b-a)f\left(a + (b-a)U_{n}\right)\right)_{n \in \mathbb{N}}$ is an independent and identically distributed sequence of random variables satisfying:
$$\frac{(b-a)\sum_{k=0}^{n}f\left(a + (b - a)U_{k}\right)}{n+1} \xrightarrow[n \to +\infty]{\mathbb{P}\text{-a.s.}} \mathbb{E}\left((b - a)f\left(a + (b - a)U\right)\right) = \int_{a}^{b}f(x)\mathrm{d}x$$
whereupon:
$$\frac{(b-a)\sum_{k=0}^{n}f\left(a + (b - a)U_{k}\right)}{n+1} \xrightarrow[n \to +\infty]{\mathbb{P}\text{-a.s.}} \int_{a}^{b}f(x)\mathrm{d}x$$

In [26]:
def monte_carlo_integral(f, a, b, n, show_performance_metrics = True):
    x = smp.symbols("x")
    assert a < b and type(n) is int
    g = smp.lambdify(x, f, 'numpy')
    start = tm.time()
    g = np.vectorize(g)
    U = np.random.uniform(0, 1, n)
    X = a + (b - a) * U
    estimate = (b - a) * np.mean(g(X))
    end = tm.time()
    computation_time = end - start
    if computation_time == 0.0:
        iterations_per_second = "\u221e"
    else:
        iterations_per_second = f"{int(n / computation_time):,}"
    integral = smp.integrate(f, (x, a, b)).evalf()
    if show_performance_metrics:
        print(f"Function: {f}")
        print(f"Domain: [{a}, {b}]")
        print(f"Evaluate: {smp.printing.latex(smp.Integral(f, (x, a, b)))}")
        print(f"Method: {monte_carlo_integral.__name__}")
        print(f"Number of iterations: {n:,}")
        print(f"Estimate: {estimate}")
        print(f"Exact Value: {integral}")
        print(f"Relative error: {100 * (estimate - integral) / integral:.2f}%")
        print(f"Computation time: {computation_time:.3f} seconds")
        print(f"Iterations per second: {iterations_per_second}")
    else:
        pass
    return estimate

monte_carlo_integral.__name__ = "Monte Carlo Method"

monte_carlo_integral(f, a, b, n, True)

Function: exp(-x**2)
Domain: [-100, 100]
Evaluate: \int\limits_{-100}^{100} e^{- x^{2}}\, dx
Method: Monte Carlo Method
Number of iterations: 1,000,000
Estimate: 1.7830374618328642
Exact Value: 1.77245385090552
Relative error: 0.60%
Computation time: 2.631 seconds
Iterations per second: 380,026


1.7830374618328642

## **Credits**
- “[Numerical Integration - Midpoint, Trapezoid, Simpson’s Rule](https://math.libretexts.org/@go/page/10269).” 2021. July 25, 2021.
- Wikipedia contributors, "[Numerical integration](https://en.wikipedia.org/w/index.php?title=Numerical_integration&oldid=1188650985)," Wikipedia, The Free Encyclopedia (accessed February 15, 2024)
- Wikipedia contributors, "[Riemann sum](https://en.wikipedia.org/w/index.php?title=Riemann_sum&oldid=1198175653)," Wikipedia, The Free Encyclopedia (accessed February 16, 2024).
- Wikipedia contributors, "[Trapezoidal rule](https://en.wikipedia.org/w/index.php?title=Trapezoidal_rule&oldid=1181282452)," Wikipedia, The Free Encyclopedia (accessed February 16, 2024).
- Wikipedia contributors, "[Simpson's rule](https://en.wikipedia.org/w/index.php?title=Simpson%27s_rule&oldid=1203533339)," Wikipedia, The Free Encyclopedia (accessed February 16, 2024).
- Wikipedia contributors, "[Monte Carlo integration](https://en.wikipedia.org/w/index.php?title=Monte_Carlo_integration&oldid=1188150594)," Wikipedia, The Free Encyclopedia (accessed February 16, 2024).
- The original code is provided as-is in this project by Zakaria Zerrouki.